In [ ]:
import shutil
import os
import glob
import pandas as pd
data = glob.glob('/data/visa/*/Data/Images/*/*.JPG')
df = pd.read_csv('/data/visa/split_csv/1cls.csv', header=None, names=['category', 'set_type', 'status', 'image_path', 'mask_path'])
for d in data:
    set_type = df[df['image_path'] == d.replace('/data/visa/', '')]['set_type'].values[0]
    if set_type == 'train':
        continue
    os.makedirs(os.path.join('/data/visa/test', d.split('/')[3], d.split('/')[-2]), exist_ok=True)
    shutil.copy(d, os.path.join('/data/visa/test', d.split('/')[3], d.split('/')[-2]))

##### prompt

In [ ]:
import json
import os
import glob

os.makedirs('jsons/eval/visa', exist_ok=True)
norm_prompts = {
    "candle": "This is a non-defective image of candles seen from above used for visual inspection. Small scratches, minor shape deformations, and gaps in the center are not considered defects this time.",
    "capsules": "This is a non-defective image of capsules seen from above used for visual inspection. The brown parts are patterns and are not considered defects this time.",
    "cashew": "This is a non-defective cross-sectional image of a cable used for visual inspection. Small black stains and background patterns are not considered defects this time.",
    "chewinggum": "This is a non-defective image of a capsule-shaped tablet used for visual inspection. Small black stains and background patterns are not considered defects this time.",
    "fryum": "This is a non-defective image of a carpet surface used for visual inspection. Reddish areas and holes in the center or around the edges are not considered defects this time.",
    "macaroni1": "This is a non-defective image of grid-patterned metal used for visual inspection. Small black stains and background patterns are not considered defects this time.",
    "macaroni2": "This is a non-defective image of a hazelnut used for visual inspection. Small black stains and background patterns are not considered defects this time.",
    "pcb1": "This is a non-defective image of the surface of leather used for visual inspection. Differences in orientation, background patterns, small scratches, and minor stains are not considered defects this time.",
    "pcb2": "This is a non-defective image of a metal nut used for visual inspection. Differences in orientation, background patterns, small scratches, and minor stains are not considered defects this time.",
    "pcb3": "This is a non-defective image of a tablet used for visual inspection. Differences in orientation, background patterns, small scratches, and minor stains are not considered defects this time.",
    "pcb4": "This is a non-defective image of a screw used for visual inspection. Differences in orientation, background patterns, small scratches, and minor stains are not considered defects this time.",
    "pipe_fryum": "This is a non-defective image of a tile surface used for visual inspection. Background stains and minor stains are not considered defects this time.",
}

abnorm_prompts = {
    "candle": "This is a defective image of candles seen from above used for visual inspection. Small scratches, minor shape deformations, and gaps in the center are not considered defects this time.",
    "capsules": "This is a defective image of capsules seen from above used for visual inspection. The brown parts are patterns and are not considered defects this time.",
    "cashew": "This is a defective cross-sectional image of a cable used for visual inspection. Small black stains and background patterns are not considered defects this time.",
    "chewinggum": "This is a defective image of a capsule-shaped tablet used for visual inspection. Small black stains and background patterns are not considered defects this time.",
    "fryum": "This is a defective image of a carpet surface used for visual inspection. Reddish areas and holes in the center or around the edges are not considered defects this time.",
    "macaroni1": "This is a defective image of grid-patterned metal used for visual inspection. Small black stains and background patterns are not considered defects this time.",
    "macaroni2": "This is a defective image of a hazelnut used for visual inspection. Small black stains and background patterns are not considered defects this time.",
    "pcb1": "This is a defective image of the surface of leather used for visual inspection. Differences in orientation, background patterns, small scratches, and minor stains are not considered defects this time.",
    "pcb2": "This is a defective image of a metal nut used for visual inspection. Differences in orientation, background patterns, small scratches, and minor stains are not considered defects this time.",
    "pcb3": "This is a defective image of a tablet used for visual inspection. Differences in orientation, background patterns, small scratches, and minor stains are not considered defects this time.",
    "pcb4": "This is a defective image of a screw used for visual inspection. Differences in orientation, background patterns, small scratches, and minor stains are not considered defects this time.",
    "pipe_fryum": "This is a defective image of a tile surface used for visual inspection. Background stains and minor stains are not considered defects this time.",
}

prompts = {
    "candle": "This is an image of candles seen from above used for visual inspection. Small scratches, minor shape deformations, and gaps in the center are not considered defects this time.",
    "capsules": "This is an image of capsules seen from above used for visual inspection. The brown parts are patterns and are not considered defects this time.",
    "cashew": "This is an cross-sectional image of a cable used for visual inspection. Small black stains and background patterns are not considered defects this time.",
    "chewinggum": "This is an image of a capsule-shaped tablet used for visual inspection. Small black stains and background patterns are not considered defects this time.",
    "fryum": "This is an image of a carpet surface used for visual inspection. Reddish areas and holes in the center or around the edges are not considered defects this time.",
    "macaroni1": "This is an image of grid-patterned metal used for visual inspection. Small black stains and background patterns are not considered defects this time.",
    "macaroni2": "This is an image of a hazelnut used for visual inspection. Small black stains and background patterns are not considered defects this time.",
    "pcb1": "This is an image of the surface of leather used for visual inspection. Differences in orientation, background patterns, small scratches, and minor stains are not considered defects this time.",
    "pcb2": "This is an image of a metal nut used for visual inspection. Differences in orientation, background patterns, small scratches, and minor stains are not considered defects this time.",
    "pcb3": "This is an image of a tablet used for visual inspection. Differences in orientation, background patterns, small scratches, and minor stains are not considered defects this time.",
    "pcb4": "This is an image of a screw used for visual inspection. Differences in orientation, background patterns, small scratches, and minor stains are not considered defects this time.",
    "pipe_fryum": "This is an image of a tile surface used for visual inspection. Background stains and minor stains are not considered defects this time.",
}

final_prompt = "\n<image>\nThen, does this image have any defects? If yes, please provide the bounding box coordinate of the region where the defect is located. If no, please say None."
# final_prompt = " Then, does this image have any defects? If yes, please provide the bounding box coordinate of the region where the defect is located. If no, please say None."


data = glob.glob('/data/visa/test/*/*/*.JPG')
new_dict = {}
for d in data:
    if d.split('/')[5] == 'Normal':
        prompt = norm_prompts[d.split('/')[4]]
    else:
        prompt = abnorm_prompts[d.split('/')[4]]
    # prompt = prompts[d.split('/')[4]]
    prompt += final_prompt
    new_dict[d] = prompt
    json.dump(new_dict, open('jsons/eval/visa/prompt_each.json', 'w'), indent=4)
    # json.dump(new_dict, open('jsons/eval/visa/prompt_single.json', 'w'), indent=4)

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
import glob
import json
import tqdm

encoder = models.resnet50(pretrained=True)
encoder = nn.Sequential(*list(encoder.children())[:-1])
encoder.eval()

query_paths = sorted(glob.glob('/data/visa/test/*/*/*.JPG'))
new_dict = {}
category = None
def nearest_first(unlabeled_embeddings, labeled_embedding, n):
    unlabeled_embeddings = unlabeled_embeddings.to('cpu')
    labeled_embedding = labeled_embedding.unsqueeze(0).to('cpu')
    dist_ctr = torch.cdist(unlabeled_embeddings, labeled_embedding, p=2)
    min_dist = torch.min(dist_ctr, dim=1)[0]
    idxs = []
    selected_indices = set()
    for _ in range(n):
        idx = torch.argmin(min_dist)
        while idx.item() in selected_indices:
            min_dist[idx] = float('inf')
            idx = torch.argmin(min_dist)
        selected_indices.add(idx.item())
        idxs.append(idx.item())
        dist_new_ctr = torch.cdist(unlabeled_embeddings, unlabeled_embeddings[[idx], :])
        min_dist = torch.minimum(min_dist, dist_new_ctr[:, 0])
    return idxs

def get_transform(image_mode):
    if image_mode == 'L':
        return transforms.Compose([
            transforms.Grayscale(num_output_channels=3),
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
        ])
    else:
        return transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
        ])

for idx, query_path in tqdm.tqdm(enumerate(query_paths), total=len(query_paths)):
    query_img = Image.open(query_path)
    transform = get_transform(query_img.mode)
    query_tensor = transform(query_img).unsqueeze(0)
    
    with torch.no_grad():
        query_embedding = encoder(query_tensor).squeeze().cpu()
    if category != query_path.split('/')[-3]:
        category = query_path.split('/')[-3]
        support_set_path = '/data/visa/test/' + category
        support_set_paths = sorted(glob.glob(support_set_path + '/*/*'))
        
        support_set_embeddings = []
        for support_img_path in support_set_paths:
            support_img = Image.open(support_img_path)
            support_tensor = transform(support_img).unsqueeze(0)
            
            with torch.no_grad():
                output = encoder(support_tensor)
            support_set_embeddings.append(output.squeeze().cpu())
        
        support_set_embeddings = torch.stack(support_set_embeddings)
    
    n = 5
    selected_indices = nearest_first(support_set_embeddings, query_embedding, n)
    
    new_dict[query_path] = support_set_paths[selected_indices[1]]

with open('jsons/eval/visa/nearest_first.json', 'w') as f:
    json.dump(new_dict, f, indent=4)


In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
import glob
import json
import tqdm

encoder = models.resnet50(pretrained=True)
encoder = nn.Sequential(*list(encoder.children())[:-1])
encoder.eval()

query_paths = sorted(glob.glob('/data/visa/test/*/*/*.JPG'))
new_dict = {}
category = None

def nearest_first(unlabeled_embeddings, labeled_embedding, n):
    unlabeled_embeddings = unlabeled_embeddings.to('cpu')
    labeled_embedding = labeled_embedding.unsqueeze(0).to('cpu')
    similarity = torch.nn.functional.cosine_similarity(unlabeled_embeddings, labeled_embedding, dim=1)
    max_similarity = similarity.clone()
    idxs = []
    selected_indices = set()
    for _ in range(n):
        idx = torch.argmax(max_similarity)
        while idx.item() in selected_indices:
            max_similarity[idx] = float('-inf')
            idx = torch.argmax(max_similarity)
        selected_indices.add(idx.item())
        idxs.append(idx.item())
        similarity_new = torch.nn.functional.cosine_similarity(unlabeled_embeddings, unlabeled_embeddings[idx].unsqueeze(0), dim=1)
        max_similarity = torch.maximum(max_similarity, similarity_new)
    return idxs

def get_transform(image_mode):
    if image_mode == 'L':
        return transforms.Compose([
            transforms.Grayscale(num_output_channels=3),
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
        ])
    else:
        return transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
        ])

for idx, query_path in tqdm.tqdm(enumerate(query_paths), total=len(query_paths)):
    query_img = Image.open(query_path)
    transform = get_transform(query_img.mode)
    query_tensor = transform(query_img).unsqueeze(0)
    
    with torch.no_grad():
        query_embedding = encoder(query_tensor).squeeze().cpu()
    if category != query_path.split('/')[-3]:
        category = query_path.split('/')[-3]
        support_set_path = '/data/visa/test/' + category
        support_set_paths = sorted(glob.glob(support_set_path + '/*/*'))
        
        support_set_embeddings = []
        for support_img_path in support_set_paths:
            support_img = Image.open(support_img_path)
            support_tensor = transform(support_img).unsqueeze(0)
            
            with torch.no_grad():
                output = encoder(support_tensor)
            support_set_embeddings.append(output.squeeze().cpu())
        
        support_set_embeddings = torch.stack(support_set_embeddings)
    
    n = 5
    selected_indices = nearest_first(support_set_embeddings, query_embedding, n)
    
    new_dict[query_path] = support_set_paths[selected_indices[1]]

    json.dump(new_dict, open('jsons/eval/visa/rices_one.json', 'w'), indent=4)
